In [1]:
import geemap
import json
import os
import requests
import re
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd
import ee
import numpy as np
from glob import glob
#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [2]:
#set up GEE map canvas (will be updated with every addLayer() method)
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)
Map.add_basemap('SATELLITE')
Map.add_basemap('CartoDB.DarkMatter')

#study area
file_path = './data/boundaries/ramsar_stlucia.geojson'
with open(file_path) as f:
    coord = json.load(f)['features'][0]['geometry']['coordinates'][0][0]
    line_geom = ee.Geometry.LineString(coord)
    
    #polygon used for clipping purposes
    poly_geom = ee.Geometry.Polygon(coord)
    
Map.addLayer(ee_object=line_geom, vis_params={'color':'red'}, name="Lesser Isimangaliso Wetland Park")
Map
    


Map(center=[-27.93186, 32.48897], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [3]:
def get_meta_s2(data_info):
    prod_id = data_info['properties']['PRODUCT_ID']
    date = pd.to_datetime(re.findall(r"(\d{8})T", prod_id)[0]).date()
    cl_cover = round(data_info['properties']['CLOUD_COVERAGE_ASSESSMENT'],2)
    tile = re.findall(r"(\d{2}[A-Z]{3})", prod_id)[0]
    df = pd.DataFrame({'id':prod_id,'date':date,'tile':tile,'cloudcover':cl_cover},index=[0])
    return df

def filter_duplicates (s2_col_list):
    s2_filtered = []
    for i in range(s2_col.size().getInfo()):
        prod_id =  s2_col_list.get(i).getInfo()['properties']['PRODUCT_ID']
        if 'OPER_PRD' not in prod_id:
            s2_filtered += [ee.Image(s2_col_list.get(i))]
    s2_filtered_col = ee.ImageCollection(s2_filtered)
    s2_filtered_list = s2_filtered_col.toList(s2_filtered_col.size())
    return s2_filtered_list

def plot_rgb(data):
    img = ee.Image(data)
    date = data.getInfo()['properties']['Date']
    viz_rgb = {'bands': ['B4', 'B3', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':30}
    Map.addLayer(img, viz_rgb, f'{date} RGB')
    
def plot_mask(data,layer_name,colour='lightblue'):
    img = ee.Image(data).select(layer_name)
    img_mask = img.updateMask(img.eq(1))
    date = data.getInfo()['properties']['Date']
    Map.addLayer(img_mask, {'palette': colour}, f'{date} {layer_name}')
    
def add_wiw(img):
    wiw = img.expression(
        '((B8A/10000) <= 0.1804) && ((B12/10000) <= 0.1131)',{'B8A':img.select('B8A'),'B12':img.select('B12')}
    ).rename('WIW')
    return img.addBands(wiw)

def add_rws(img):
    mndwi = img.normalizedDifference(['B3','B11']).rename('MNDWI')
    mgrn = img.select(['B4','B3','B8']).reduce(ee.Reducer.min()).rename('MGRN')
    rws = img.expression(
        'MNDWI > 0.3 && (MGRN/10000) < 0.15',{'MNDWI':mndwi,'MGRN':mgrn}
    ).rename('RWS')
    return img.addBands(rws)

def apply_mask(image):
    mask = image.select('RWS').eq(1)
    rgb = image.select(['B4', 'B3', 'B2','RWS','WIW'])
    masked_rgb = rgb.updateMask(mask)
    return masked_rgb

def add_kmeans(image):
    rgb = image.select(['B4', 'B3', 'B2'])
    train_samples = rgb.sample(scale=10,numPixels=100,seed=42)
    k_means = ee.Clusterer.wekaKMeans(3).train(train_samples)
    img_cluster = rgb.cluster(k_means)
    
    return image.addBands(img_cluster)

In [ ]:
#get Sentinel-2 Level-1C collection for 2016

cl_pct = 0

s2_col = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(line_geom).filterDate('2016-01-01','2016-12-31') \
    .filter(ee.Filter.inList('MGRS_TILE', ['36JVP','36JVQ'])) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cl_pct)) \
    .select(['B2', 'B3', 'B4','B8','B8A','B11','B12'])\

#filter duplicated products 
s2_col_list = filter_duplicates(s2_col.toList(s2_col.size()))

print('Nr. images found: ', s2_col_list.size().getInfo())

#store product information
# products = list(map(get_meta_s2,s2_col_list.getInfo()))
# products_df = pd.concat(products).reset_index(drop=True)


In [ ]:
#find matching tiles by date
tile_matches = {}
for i in range(s2_col_list.size().getInfo()):
    data = s2_col_list.get(i)
    prod_id = data.getInfo()['properties']['PRODUCT_ID']
    date = str(pd.to_datetime(re.findall(r"(\d{8})T", prod_id)[0]).date())
    tile_matches.setdefault(date,[]).append(ee.Image(data))

#create a mosaic from matched tiles
s2_mosaics = []
for date in tile_matches:
    match = tile_matches[date]
    if len(match)==2:
        img_mos = ee.ImageCollection(match).mosaic().clip(poly_geom)
        s2_mosaics += [img_mos.set({'Date': date})]
    else:
        img = match[0].clip(poly_geom)
        s2_mosaics += [img.set({'Date': date})]
s2_mos_col = ee.ImageCollection(s2_mosaics)

print('Nr. mosaics: ', s2_mos_col .size().getInfo())

In [ ]:
#add indices
s2_mos_stack = s2_mos_col.map(add_rws)
s2_mos_stack = s2_mos_stack.map(add_wiw)
s2_mos_stack_list  = s2_mos_stack.toList(s2_mos_stack.size())

In [ ]:
#apply mask
s2_masked_stack = s2_mos_stack.map(apply_mask)
s2_masked_stack_list = s2_masked_stack.toList(s2_masked_stack.size())

In [ ]:
#add kmeans
s2_masked_stack_kmeans = s2_masked_stack.map(add_kmeans)
s2_masked_stack_kmeans_list = s2_masked_stack_kmeans.toList(s2_masked_stack_kmeans.size())

In [ ]:
#plot latest available data
data = s2_masked_stack_kmeans_list.get(-1)
img = ee.Image(data).select('cluster')
rgb = ee.Image(data).select(['B4','B3','B2'])
# Map.addLayer(img.randomVisualizer(), {}, 'all')

In [ ]:
img_all = img.reduceToVectors(scale=30,bestEffort=True)
img_0 = img_all.filter(ee.Filter.eq('label', 0)).union(maxError=10).toList(1).get(0)
img_1 = img_all.filter(ee.Filter.eq('label', 1)).union(maxError=10).toList(1).get(0)
img_2 = img_all.filter(ee.Filter.eq('label', 2)).union(maxError=10).toList(1).get(0)
img_all2 = ee.FeatureCollection([img_0,img_1,img_2])

In [ ]:
reducers = ee.Reducer.mean().combine(reducer2 = ee.Reducer.stdDev(),sharedInputs= True)
stats = rgb.reduceRegions(
  collection= img_all2,
  reducer= reducers,
  scale= 30)

stats_list = stats.toList(stats.size())

In [ ]:
stats_list.get(0).getInfo()['properties']

In [ ]:
# #convert mask to polygon explicitly 
# img = ee.Image(data).select('RWS')
# img_mask = img.updateMask(img.eq(1)).reduceToVectors(scale=10,bestEffort=True)
# polygon = img_mask.union(maxError=10).geometry()

# test = img_cluster.updateMask(img_cluster.eq(2))
# Map.addLayer(test.randomVisualizer(), {}, 'test 2')